# IntelliSort - Waste Classification Training

## Overview
This notebook trains a YOLOv8s-cls model to classify waste into 7 categories:
- **Recyclables:** Plastic, Glass, Metal, Paper, Cardboard
- **Compost:** Organic
- **Landfill:** Trash

---

## Training Pipeline
1. **Data Preparation** - Organize and merge datasets
2. **Model Training** - Train YOLOv8s-cls (80 epochs, ~4 hours)
3. **Evaluation** - Test accuracy and per-class metrics

---

## Prerequisites
- **GPU:** NVIDIA with 6GB+ VRAM (CUDA 11.8+)
- **Datasets:** 
  - Dataset 1: `data/waste_classification/dataset/` (30 classes)
  -https://www.kaggle.com/datasets/alistairking/recyclable-and-household-waste-classification
  - Dataset 2: `data/waste_simple/DATASET/` (Organic/Recyclable)
  -https://www.kaggle.com/datasets/techsash/waste-classification-data
- **Packages:** `ultralytics`, `torch`, `numpy<2`, `opencv-python`

---

## Getting Started
Run cells sequentially. Total runtime: ~5-6 hours.

---

# 1. Setup & Imports

In [25]:
import os
import shutil
import json
import random
from pathlib import Path
from collections import Counter
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

if 'model_work' in os.getcwd():
    os.chdir('..')

print(f"Working directory: {os.getcwd()}")
print("Setup complete")

Working directory: f:\shiii\multidiciplinary\intellisort
Setup complete


In [ ]:
CLASS_MAPPING = {
    'plastic': [
        'plastic_cup_lids',
        'plastic_detergent_bottles',
        'plastic_food_containers',
        'plastic_shopping_bags',
        'plastic_soda_bottles',
        'plastic_straws',
        'plastic_water_bottles',
        'disposable_plastic_cutlery',
        'plastic_film',
    ],
    
    'glass': [
        'glass_beverage_bottles',
        'glass_cosmetic_containers',
        'glass_food_jars',
    ],
    
    'metal': [
        'aerosol_cans',
        'aluminum_food_cans',
        'aluminum_soda_cans',
        'metal_bottle_caps',
    ],
    
    'paper': [
        'magazines',
        'newspaper',
        'office_paper',
        'paper_cups',
    ],
    
    'cardboard': [
        'cardboard_boxes',
        'cardboard_packaging',
    ],
    
    'organic': [
        'coffee_grounds',
        'eggshells',
        'food_waste',
    ],
    
    'trash': [
        'styrofoam_food_containers',
        'clothing',
        'shoes',
    ]
}

DISPOSAL_INFO = {
    'plastic': {
        'disposal': 'Recyclable',
        'bin_color': 'Blue/Yellow',
        'tip': 'Rinse containers and check recycling number (1, 2, 5 typically accepted)'
    },
    'glass': {
        'disposal': 'Recyclable',
        'bin_color': 'Green/Blue',
        'tip': 'Remove caps, rinse thoroughly. Glass can be recycled infinitely!'
    },
    'metal': {
        'disposal': 'Recyclable',
        'bin_color': 'Blue',
        'tip': 'Rinse cans and crush to save space'
    },
    'paper': {
        'disposal': 'Recyclable',
        'bin_color': 'Blue',
        'tip': 'Keep dry and clean. Remove any plastic coating'
    },
    'cardboard': {
        'disposal': 'Recyclable',
        'bin_color': 'Blue',
        'tip': 'Flatten boxes. Remove tape and labels if possible'
    },
    'organic': {
        'disposal': 'Compost',
        'bin_color': 'Brown/Green',
        'tip': 'Great for composting! Creates nutrient-rich soil'
    },
    'trash': {
        'disposal': 'Landfill',
        'bin_color': 'Black/Gray',
        'tip': 'Cannot be recycled. Consider reducing usage'
    }
}

Path('model_work').mkdir(exist_ok=True)
with open('model_work/class_mapping.json', 'w') as f:
    json.dump({'mapping': CLASS_MAPPING, 'disposal': DISPOSAL_INFO}, f, indent=2)

print(f"Defined {len(CLASS_MAPPING)} custom categories")
print(f"Mapped {sum(len(v) for v in CLASS_MAPPING.values())} original classes")

---

# 6. Merge Datasets into Combined Dataset

Combines both datasets and maps to 7 final categories.

## Input:
- **Dataset 1:** `data/waste_classification/organized/` (30 classes)
- **Dataset 2:** `data/waste_simple/DATASET/` (Organic/Recyclable)

## Output:
- **Combined:** `data/combined_dataset/` (7 classes)

## Process:
1. Maps 30 classes → 7 categories
2. Adds Dataset 2 organic images → `organic/` class
3. Adds Dataset 2 recyclable images → `plastic/` class
4. Prefixes filenames to avoid conflicts (`d1_`, `d2_`)
5. Saves config to `model_work/dataset_config.pkl`

## Expected Final Dataset:
| Class | Train | Val | Test | Total |
|-------|-------|-----|------|-------|
| cardboard | 496 | 111 | 236 | 843 |
| glass | 744 | 0 | 354 | 1,098 |
| metal | 744 | 0 | 354 | 1,098 |
| organic | 13,309 | 0 | 354 | 13,663 |
| paper | 992 | 0 | 472 | 1,464 |
| plastic | 11,983 | 0 | 944 | 12,927 |
| trash | 744 | 0 | 354 | 1,098 |
| **TOTAL** | **29,012** | **5,607** | **3,068** | **37,687** |

⚠️ **Note:** Dataset is imbalanced (organic: 46%, plastic: 41%)

In [ ]:
import shutil
import random
from pathlib import Path
from tqdm import tqdm

dataset1_raw = Path('data/waste_classification/dataset')
dataset1_organized = Path('data/waste_classification/organized')


print("ORGANIZING DATASET 1")


if not dataset1_raw.exists():
    print(f"Raw dataset not found at: {dataset1_raw}")
    print("   Make sure you to have dataset folder")
else:
    print(f"Found raw dataset: {dataset1_raw}\n")
    
    class_folders = sorted([d for d in dataset1_raw.iterdir() if d.is_dir()])
    print(f"Found {len(class_folders)} class folders\n")
    
    for split in ['train', 'val', 'test']:
        for cls_folder in class_folders:
            (dataset1_organized / split / cls_folder.name).mkdir(parents=True, exist_ok=True)
    
    print("Splitting images into train/val/test (70/15/15)...\n")
    
    stats = {'train': 0, 'val': 0, 'test': 0}
    
    for cls_folder in tqdm(class_folders, desc="Processing classes"):
        cls_name = cls_folder.name
        
        images = list(cls_folder.glob('*.jpg')) + \
                 list(cls_folder.glob('*.png')) + \
                 list(cls_folder.glob('*.jpeg')) + \
                 list(cls_folder.glob('*.JPG')) + \
                 list(cls_folder.glob('*.PNG'))
        
        if len(images) == 0:
            continue
        
        random.seed(42)
        random.shuffle(images)
        
        if len(images) >= 10:
            # 70-15-15 split
            train_idx = int(len(images) * 0.7)
            val_idx = int(len(images) * 0.85)
            
            train_imgs = images[:train_idx]
            val_imgs = images[train_idx:val_idx]
            test_imgs = images[val_idx:]
        elif len(images) >= 3:
            train_idx = int(len(images) * 0.7)
            train_imgs = images[:train_idx]
            val_imgs = images[train_idx:]
            test_imgs = []
        else:
            train_imgs = images
            val_imgs = []
            test_imgs = []
        
        for img in train_imgs:
            dest = dataset1_organized / 'train' / cls_name / img.name
            shutil.copy2(img, dest)
            stats['train'] += 1
        
        for img in val_imgs:
            dest = dataset1_organized / 'val' / cls_name / img.name
            shutil.copy2(img, dest)
            stats['val'] += 1
        
        for img in test_imgs:
            dest = dataset1_organized / 'test' / cls_name / img.name
            shutil.copy2(img, dest)
            stats['test'] += 1
    
    print("\n" + "="*70)
    print("DATASET 1 ORGANIZED!")
    
    print(f"\nTrain: {stats['train']:,} images")
    print(f"Val:   {stats['val']:,} images")
    print(f"Test:  {stats['test']:,} images")
    print(f"Total: {sum(stats.values()):,} images")
    
    print(f"\nOrganized dataset saved to:")
    print(f"   {dataset1_organized}")

print("\n" + "="*70)

In [ ]:
import shutil
import random
from pathlib import Path
from tqdm import tqdm

dataset1_raw = Path('data/waste_classification/dataset')
dataset1_organized = Path('data/waste_classification/organized')


print("ORGANIZING DATASET 1 - WITH SUBFOLDERS")

if not dataset1_raw.exists():
    print(f"Raw dataset not found!")
else:
    print(f"Source: {dataset1_raw}")
    print(f"Target: {dataset1_organized}\n")
    
    class_folders = sorted([d for d in dataset1_raw.iterdir() if d.is_dir()])
    print(f"Found {len(class_folders)} class folders\n")
    
    print("Creating directories...")
    for split in ['train', 'val', 'test']:
        for cls_folder in class_folders:
            (dataset1_organized / split / cls_folder.name).mkdir(parents=True, exist_ok=True)
    print("Directories created\n")
    
    stats = {'train': 0, 'val': 0, 'test': 0}
    
    print("Copying images from subfolders...\n")
    
    for cls_folder in tqdm(class_folders, desc="Processing classes"):
        cls_name = cls_folder.name
        
        images = []
        
        subfolders = [d for d in cls_folder.iterdir() if d.is_dir()]
        
        if len(subfolders) > 0:
            for subfolder in subfolders:
                for ext in ['*.jpg', '*.jpeg', '*.png', '*.JPG', '*.JPEG', '*.PNG']:
                    images.extend(list(subfolder.glob(ext)))
        else:
            for ext in ['*.jpg', '*.jpeg', '*.png', '*.JPG', '*.JPEG', '*.PNG']:
                images.extend(list(cls_folder.glob(ext)))
        
        if len(images) == 0:
            continue
        
        random.seed(42)
        random.shuffle(images)
        
        if len(images) >= 10:
            train_idx = int(len(images) * 0.7)
            val_idx = int(len(images) * 0.85)
            train_imgs = images[:train_idx]
            val_imgs = images[train_idx:val_idx]
            test_imgs = images[val_idx:]
        elif len(images) >= 3:
            train_idx = int(len(images) * 0.7)
            train_imgs = images[:train_idx]
            val_imgs = images[train_idx:]
            test_imgs = []
        else:
            train_imgs = images
            val_imgs = []
            test_imgs = []
        
        for img in train_imgs:
            dest = dataset1_organized / 'train' / cls_name / img.name
            shutil.copy2(img, dest)
            stats['train'] += 1
        
        for img in val_imgs:
            dest = dataset1_organized / 'val' / cls_name / img.name
            shutil.copy2(img, dest)
            stats['val'] += 1
        
        for img in test_imgs:
            dest = dataset1_organized / 'test' / cls_name / img.name
            shutil.copy2(img, dest)
            stats['test'] += 1
    
    print("\n" + "="*70)
    print("ORGANIZATION COMPLETE!")
    
    print(f"\nTrain: {stats['train']:,} images")
    print(f"Val:   {stats['val']:,} images")
    print(f"Test:  {stats['test']:,} images")
    print(f"Total: {sum(stats.values()):,} images")
    
    if sum(stats.values()) > 0:
        print(f"\n✅ Files copied successfully!")
    else:
        print(f"\n❌ No files copied - check dataset structure")

print("\n" + "="*70)

In [ ]:
from pathlib import Path

organized_path = Path('../data/waste_classification/organized')


print("VERIFICATION")


for split in ['train', 'val', 'test']:
    split_path = organized_path / split
    
    if not split_path.exists():
        print(f"\n{split}: Not found")
        continue
    
    folders = sorted([d for d in split_path.iterdir() if d.is_dir()])
    total_images = sum(len(list(f.glob('*.jpg'))) + len(list(f.glob('*.png'))) 
                      for f in folders)
    
    print(f"\n{split.upper()}:")
    print(f"   Folders: {len(folders)}")
    print(f"   Images:  {total_images:,}")
    
    if len(folders) > 0:
        print(f"   Samples:")
        for folder in folders[:3]:
            img_count = len(list(folder.glob('*.jpg'))) + len(list(folder.glob('*.png')))
            print(f"      {folder.name}: {img_count} images")

print("\n" + "="*70)

VERIFICATION

TRAIN:
   Folders: 30
   Images:  7,440
   Samples:
      aerosol_cans: 248 images
      aluminum_food_cans: 248 images
      aluminum_soda_cans: 248 images

VAL:
   Folders: 30
   Images:  3,570
   Samples:
      aerosol_cans: 119 images
      aluminum_food_cans: 119 images
      aluminum_soda_cans: 119 images

TEST:
   Folders: 30
   Images:  3,540
   Samples:
      aerosol_cans: 118 images
      aluminum_food_cans: 118 images
      aluminum_soda_cans: 118 images



In [ ]:
import shutil
from pathlib import Path
from tqdm import tqdm

dataset1_organized = Path('data/waste_classification/organized')
dataset2_full = Path('data/waste_simple/DATASET')
combined_path = Path('data/combined_dataset')


print("MERGING DATASETS INTO COMBINED")


print("\nChecking datasets...")
d1_exists = dataset1_organized.exists() and (dataset1_organized / 'train').exists()
d2_exists = dataset2_full.exists()

print(f"   Dataset 1 (organized): {'✅' if d1_exists else '❌'}")
print(f"   Dataset 2 (simple):    {'✅' if d2_exists else '❌'}")

if not d1_exists:
    print("\nDataset 1 not organized yet! Run organization cell first.")
else:
    # Remove old combined dataset if exists
    if combined_path.exists():
        print(f"\nRemoving old combined dataset...")
        shutil.rmtree(combined_path)
        print("   ✅ Removed")
    
    print(f"\nCreating combined dataset structure...\n")
    
    for split in ['train', 'val', 'test']:
        for category in CLASS_MAPPING.keys():
            (combined_path / split / category).mkdir(parents=True, exist_ok=True)
    
    print("Folders created\n")
    
    stats = {cat: {'train': 0, 'val': 0, 'test': 0} for cat in CLASS_MAPPING.keys()}
    
    reverse_map = {}
    for custom_cat, orig_classes in CLASS_MAPPING.items():
        for orig in orig_classes:
            reverse_map[orig] = custom_cat
    
    
    print("PART 1: PROCESSING DATASET 1")
    
    
    for split in ['train', 'val', 'test']:
        split_path = dataset1_organized / split
        
        if not split_path.exists():
            print(f"\n{split} folder not found, skipping")
            continue
        
        print(f"\nProcessing {split}...")
        
        class_folders = sorted([d for d in split_path.iterdir() if d.is_dir()])
        
        for cls_folder in tqdm(class_folders, desc=f"  {split}"):
            orig_class = cls_folder.name
            
            if orig_class not in reverse_map:
                continue
            
            custom_cat = reverse_map[orig_class]
            
            images = list(cls_folder.glob('*.jpg')) + \
                     list(cls_folder.glob('*.png')) + \
                     list(cls_folder.glob('*.jpeg'))
            
            for img in images:
                dest_name = f"d1_{orig_class}_{img.name}"
                dest = combined_path / split / custom_cat / dest_name
                
                try:
                    shutil.copy2(img, dest)
                    stats[custom_cat][split] += 1
                except Exception as e:
                    print(f"\nError copying {img.name}: {e}")
    
    if d2_exists:
        print("\n" + "="*70)
        print("PART 2: PROCESSING DATASET 2")
        
        
        train_path = dataset2_full / 'TRAIN'
        if train_path.exists():
            print(f"\n📦 Processing TRAIN...")
            
            o_path = train_path / 'O'
            if o_path.exists():
                images = list(o_path.glob('*.jpg')) + list(o_path.glob('*.png'))
                for img in tqdm(images, desc="  Organic"):
                    dest = combined_path / 'train' / 'organic' / f"d2_o_{img.name}"
                    shutil.copy2(img, dest)
                    stats['organic']['train'] += 1
            
            r_path = train_path / 'R'
            if r_path.exists():
                images = list(r_path.glob('*.jpg')) + list(r_path.glob('*.png'))
                for img in tqdm(images, desc="  Recyclable"):
                    dest = combined_path / 'train' / 'plastic' / f"d2_r_{img.name}"
                    shutil.copy2(img, dest)
                    stats['plastic']['train'] += 1
        
        test_path = dataset2_full / 'TEST'
        if test_path.exists():
            print(f"\n📦 Processing TEST...")
            
            o_path = test_path / 'O'
            if o_path.exists():
                images = list(o_path.glob('*.jpg')) + list(o_path.glob('*.png'))
                for img in tqdm(images, desc="  Organic"):
                    dest = combined_path / 'val' / 'organic' / f"d2_o_val_{img.name}"
                    shutil.copy2(img, dest)
                    stats['organic']['val'] += 1
            
            r_path = test_path / 'R'
            if r_path.exists():
                images = list(r_path.glob('*.jpg')) + list(r_path.glob('*.png'))
                for img in tqdm(images, desc="  Recyclable"):
                    dest = combined_path / 'val' / 'plastic' / f"d2_r_val_{img.name}"
                    shutil.copy2(img, dest)
                    stats['plastic']['val'] += 1
    
    print("\n" + "="*70)
    print("COMBINED DATASET STATISTICS")
    
    print(f"\n{'Category':<12} {'Train':>8} {'Val':>8} {'Test':>8} {'Total':>8}")
    print("-"*70)
    
    for cat in sorted(CLASS_MAPPING.keys()):
        t = stats[cat]['train']
        v = stats[cat]['val']
        te = stats[cat]['test']
        total = t + v + te
        print(f"{cat:<12} {t:>8,} {v:>8,} {te:>8,} {total:>8,}")
    
    print("-"*70)
    
    total_train = sum(s['train'] for s in stats.values())
    total_val = sum(s['val'] for s in stats.values())
    total_test = sum(s['test'] for s in stats.values())
    grand_total = total_train + total_val + total_test
    
    print(f"{'TOTAL':<12} {total_train:>8,} {total_val:>8,} {total_test:>8,} {grand_total:>8,}")
    
    
    import pickle
    
    config = {
        'dataset_path': str(combined_path),
        'num_classes': len(CLASS_MAPPING),
        'class_names': list(CLASS_MAPPING.keys()),
        'class_mapping': CLASS_MAPPING,
        'disposal_info': DISPOSAL_INFO
    }
    
    Path('model_work').mkdir(exist_ok=True)
    with open('model_work/dataset_config.pkl', 'wb') as f:
        pickle.dump(config, f)
    
    print(f"\nCombined dataset created!")
    print(f"Location: {combined_path}")
    print(f"Config saved: model_work/dataset_config.pkl")
    print(f"\nReady for training")

print("\n" + "="*70)

MERGING DATASETS INTO COMBINED

📋 Checking datasets...
   Dataset 1 (organized): ❌
   Dataset 2 (simple):    ❌

❌ Dataset 1 not organized yet! Run organization cell first.



# Model Training:

In [ ]:
import os
import pickle
from pathlib import Path

# Force NVIDIA GPU
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'

import torch
from ultralytics import YOLO
import warnings
warnings.filterwarnings('ignore')

# Change to project root
if 'model_work' in os.getcwd():
    os.chdir('..')


print("INTELLISORT - TRAINING SETUP")


# GPU Check
print("\nHardware:")
if torch.cuda.is_available():
    device = 'cuda:0'
    torch.cuda.set_device(0)
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   CUDA: {torch.version.cuda}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    device = 'cpu'
    print(f"   CPU only - training will be very slow!")

# Load config
with open('model_work/dataset_config.pkl', 'rb') as f:
    config = pickle.load(f)

dataset_path = Path(config['dataset_path'])
num_classes = config['num_classes']
class_names = config['class_names']

print(f"\n Dataset:")
print(f"   Path: {dataset_path}")
print(f"   Classes: {num_classes}")
print(f"   Categories: {', '.join(class_names)}")

# Get class counts for reference
train_path = dataset_path / 'train'
class_counts = {}
for cat in class_names:
    cat_path = train_path / cat
    if cat_path.exists():
        count = len(list(cat_path.glob('*.jpg'))) + len(list(cat_path.glob('*.png')))
        class_counts[cat] = count

globals()['class_counts'] = class_counts

print(f"\nSetup complete - Ready to train")

INTELLISORT - TRAINING SETUP

🖥️ Hardware:
   ✅ GPU: NVIDIA GeForce RTX 4050 Laptop GPU
   CUDA: 11.8
   Memory: 6.44 GB

📊 Dataset:
   Path: data\combined_dataset
   Classes: 7
   Categories: plastic, glass, metal, paper, cardboard, organic, trash

✅ Setup complete - Ready to train!


In [ ]:

print("LOADING YOLO MODEL")


# Load YOLOv8s-cls
model = YOLO('yolov8s-cls.pt')

print("\n✅ YOLOv8s-cls loaded")
print("   Pre-trained on ImageNet")
print("   Model size: ~25MB")

LOADING YOLO MODEL

✅ YOLOv8s-cls loaded
   Pre-trained on ImageNet
   Model size: ~25MB


In [ ]:
# Optimized hyperparameters
EPOCHS = 80
BATCH_SIZE = 32
IMAGE_SIZE = 224
PATIENCE = 15
LEARNING_RATE = 0.0005
CHECKPOINT_FREQ = 10


print("TRAINING CONFIGURATION")


print(f"\n📋 Hyperparameters:")
print(f"   Epochs:           {EPOCHS}")
print(f"   Batch size:       {BATCH_SIZE}")
print(f"   Image size:       {IMAGE_SIZE}×{IMAGE_SIZE}")
print(f"   Learning rate:    {LEARNING_RATE}")
print(f"   Patience:         {PATIENCE} epochs")
print(f"   Checkpoints:      Every {CHECKPOINT_FREQ} epochs")
print(f"   Device:           {device}")

# Correct time estimation
train_images = 29012
batches_per_epoch = train_images // BATCH_SIZE  # 906 batches

print(f"\n⏱️ Time Estimate:")
print(f"   Training images:   {train_images:,}")
print(f"   Batches per epoch: {batches_per_epoch}")

if device == 'cuda:0':
    seconds_per_batch = 0.3
    minutes_per_epoch = (batches_per_epoch * seconds_per_batch) / 60
    total_hours = (minutes_per_epoch * EPOCHS) / 60
    expected_hours = (minutes_per_epoch * 55) / 60  # With early stopping
    
    print(f"   Time per epoch:    ~{minutes_per_epoch:.1f} min")
    print(f"   Max time (80 epochs): ~{total_hours:.1f} hours")
    print(f"   Expected (early stop): ~{expected_hours:.1f} hours")
else:
    print(f"   ⚠️ CPU training would take 8-12 hours!")

print(f"\n⚠️ Dataset is imbalanced - will monitor per-class metrics")
print("\n✅ Configuration ready!")

TRAINING CONFIGURATION

📋 Hyperparameters:
   Epochs:           80
   Batch size:       32
   Image size:       224×224
   Learning rate:    0.0005
   Patience:         15 epochs
   Checkpoints:      Every 10 epochs
   Device:           cuda:0

⏱️ Time Estimate:
   Training images:   29,012
   Batches per epoch: 906
   Time per epoch:    ~4.5 min
   Max time (80 epochs): ~6.0 hours
   Expected (early stop): ~4.2 hours

⚠️ Dataset is imbalanced - will monitor per-class metrics

✅ Configuration ready!


In [ ]:
print("\n" + "="*70)
print("🚀 STARTING TRAINING")


print(f"\nTraining on: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")
print(f"Images: 29,012 train / 5,607 val / 3,068 test")
print(f"\n⏳ Estimated time: 3-4 hours")
print(f"💡 You can monitor GPU usage: nvidia-smi -l 1\n")

# Create output directory
output_dir = Path('model_work/runs')
output_dir.mkdir(parents=True, exist_ok=True)

# TRAIN - with AMP disabled to avoid NumPy issue
results = model.train(
    # Data
    data=str(dataset_path),
    
    # Training duration
    epochs=EPOCHS,
    patience=PATIENCE,
    
    # Image settings
    imgsz=IMAGE_SIZE,
    batch=BATCH_SIZE,
    
    # Device
    device=0,
    
    # Learning rate (optimized for fine-tuning)
    lr0=LEARNING_RATE,
    lrf=0.01,
    momentum=0.937,
    weight_decay=0.0005,
    
    # Optimizer
    optimizer='Adam',
    
    # Data augmentation
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=10,
    translate=0.1,
    scale=0.5,
    fliplr=0.5,
    flipud=0.2,
    mosaic=0.0,
    
    # Output
    project='model_work/runs',
    name='waste_classifier',
    exist_ok=True,
    
    # Saving
    save=True,
    save_period=CHECKPOINT_FREQ,
    
    # Performance
    workers=4,
    amp=False,  # ← DISABLED to avoid NumPy error (15% slower but works)
    
    # Logging
    verbose=True,
    plots=True,
    pretrained=True,
    val=True,
)

print("\n" + "="*70)
print("✅ TRAINING COMPLETE!")


# Save paths
best_model_path = Path(results.save_dir) / 'weights' / 'best.pt'
last_model_path = Path(results.save_dir) / 'weights' / 'last.pt'
results_dir = Path(results.save_dir)

globals()['best_model_path'] = best_model_path
globals()['last_model_path'] = last_model_path
globals()['results_dir'] = results_dir

print(f"\n💾 Models saved:")
print(f"   Best: {best_model_path}")
print(f"   Last: {last_model_path}")


🚀 STARTING TRAINING

Training on: NVIDIA GeForce RTX 4050 Laptop GPU
Images: 29,012 train / 5,607 val / 3,068 test

⏳ Estimated time: 3-4 hours
💡 You can monitor GPU usage: nvidia-smi -l 1

Ultralytics 8.4.11  Python-3.10.11 torch-2.1.2+cu118 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
engine\trainer: agnostic_nms=False, amp=False, angle=1.0, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data\combined_dataset, degrees=10, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=80, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.2, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0005, lrf=0.01, mask_ratio=4, max_det=300, m

: 

# Model Evaluation

In [15]:
import pickle
from pathlib import Path
from ultralytics import YOLO

# Load config from exact location
config_path = Path('dataset_config.pkl')

with open(config_path, 'rb') as f:
    config = pickle.load(f)

print("✅ Config loaded!")
print(f"   Classes: {config['class_names']}")
print(f"   Dataset: {config['dataset_path']}")

# Load model from exact location
best_model_path = Path('best.pt')
model = YOLO(best_model_path)

print(f"\n✅ Model loaded: {best_model_path.stat().st_size / 1e6:.1f} MB")

# Extract variables
dataset_path = Path('../data/combined_dataset')
class_names = config['class_names']
print(dataset_path)
# Set globals for next cells
globals()['model'] = model
globals()['dataset_path'] = dataset_path
globals()['class_names'] = class_names
globals()['config'] = config
globals()['best_model_path'] = best_model_path

print("\n✅ Ready for evaluation!")

✅ Config loaded!
   Classes: ['plastic', 'glass', 'metal', 'paper', 'cardboard', 'organic', 'trash']
   Dataset: data\combined_dataset

✅ Model loaded: 10.3 MB
..\data\combined_dataset

✅ Ready for evaluation!


In [ ]:

print("TEST SET VALIDATION")


print(f"\n📊 Running comprehensive validation on test set...")
print(f"   Dataset: {dataset_path}")
print(f"   Test images: 3,068\n")

# Run validation on test set
test_results = model.val(
    data=str(dataset_path),
    split='test',
    batch=32,
    imgsz=224,
    device=0,
    plots=True,
    verbose=True
)

print("\n" + "="*70)
print("OVERALL TEST RESULTS")


print(f"\n🎯 Accuracy Metrics:")
print(f"   Top-1 Accuracy: {test_results.top1:.4f} ({test_results.top1*100:.2f}%)")
print(f"   Top-5 Accuracy: {test_results.top5:.4f} ({test_results.top5*100:.2f}%)")

# Performance rating
if test_results.top1 >= 0.90:
    rating = "🌟 EXCELLENT"
elif test_results.top1 >= 0.85:
    rating = "✅ GOOD"
elif test_results.top1 >= 0.80:
    rating = "👍 ACCEPTABLE"
else:
    rating = "⚠️ NEEDS IMPROVEMENT"

print(f"\n   Overall Rating: {rating}")

# Save for later cells
globals()['test_results'] = test_results

print("\n✅ Test validation complete!")

TEST SET VALIDATION

📊 Running comprehensive validation on test set...
   Dataset: ..\data\combined_dataset
   Test images: 3,068

Ultralytics 8.4.11  Python-3.10.11 torch-2.1.2+cu118 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
YOLOv8s-cls summary (fused): 30 layers, 5,084,167 parameters, 0 gradients, 12.5 GFLOPs
train: F:\shiii\multidiciplinary\intellisort\data\combined_dataset\train... found 29012 images in 7 classes  
val: F:\shiii\multidiciplinary\intellisort\data\combined_dataset\val... found 5607 images in 7 classes  
test: F:\shiii\multidiciplinary\intellisort\data\combined_dataset\test... found 3068 images in 7 classes  
test: Fast image access  (ping: 0.20.1 ms, read: 7.44.9 MB/s, size: 54.2 KB)
test: Scanning F:\shiii\multidiciplinary\intellisort\data\combined_dataset\test... 3068 images, 0 corrupt: 100% ━━━━━━━━━━━━ 3068/3068 928.4it/s 3.3s0.1s
test: New cache created: F:\shiii\multidiciplinary\intellisort\data\combined_dataset\test.cache
               classes   to

In [ ]:
from collections import defaultdict
from tqdm import tqdm
import numpy as np


print("DETAILED PER-CLASS ANALYSIS")


test_path = dataset_path / 'test'

# Data structures
class_correct = defaultdict(int)
class_total = defaultdict(int)
class_predictions = defaultdict(lambda: defaultdict(int))
class_confidences = defaultdict(list)
class_top5_correct = defaultdict(int)

print(f"\n📊 Analyzing {len(class_names)} classes in detail...\n")

# Evaluate each class
for cat in class_names:
    cat_path = test_path / cat
    if not cat_path.exists():
        continue
    
    images = list(cat_path.glob('*.jpg')) + list(cat_path.glob('*.png'))
    
    for img_path in tqdm(images, desc=f"Testing {cat:12s}", leave=False):
        try:
            pred = model(img_path, verbose=False)
            
            # Get predictions
            pred_class = pred[0].names[pred[0].probs.top1]
            confidence = pred[0].probs.top1conf.item()
            top5_classes = [pred[0].names[i] for i in pred[0].probs.top5]
            
            # Record
            class_total[cat] += 1
            class_predictions[cat][pred_class] += 1
            class_confidences[cat].append(confidence)
            
            # Top-1 accuracy
            if pred_class == cat:
                class_correct[cat] += 1
            
            # Top-5 accuracy
            if cat in top5_classes:
                class_top5_correct[cat] += 1
                
        except Exception as e:
            continue

# Display results
print("\n" + "="*70)
print("PER-CLASS PERFORMANCE REPORT")


print(f"\n{'Category':<12} {'Test Imgs':>10} {'Top-1':>8} {'Top-5':>8} {'Avg Conf':>10} {'Status'}")
print("-"*90)

class_metrics = {}

for cat in sorted(class_names):
    if cat not in class_total or class_total[cat] == 0:
        continue
    
    total = class_total[cat]
    correct = class_correct[cat]
    top5_correct = class_top5_correct[cat]
    
    # Metrics
    top1_acc = (correct / total) * 100
    top5_acc = (top5_correct / total) * 100
    avg_conf = np.mean(class_confidences[cat]) * 100
    
    # Store
    class_metrics[cat] = {
        'total': total,
        'correct': correct,
        'top1_accuracy': top1_acc,
        'top5_accuracy': top5_acc,
        'avg_confidence': avg_conf
    }
    
    # Status
    if top1_acc >= 95:
        status = "🌟 Excellent"
    elif top1_acc >= 90:
        status = "✅ Great"
    elif top1_acc >= 85:
        status = "👍 Good"
    elif top1_acc >= 80:
        status = "⚠️ Fair"
    else:
        status = "❌ Poor"
    
    print(f"{cat:<12} {total:>10,} {top1_acc:>7.1f}% {top5_acc:>7.1f}% {avg_conf:>9.1f}%  {status}")

print("-"*90)

# Overall
total_correct = sum(class_correct.values())
total_images = sum(class_total.values())
total_top5 = sum(class_top5_correct.values())
overall_top1 = (total_correct / total_images) * 100
overall_top5 = (total_top5 / total_images) * 100

print(f"{'OVERALL':<12} {total_images:>10,} {overall_top1:>7.1f}% {overall_top5:>7.1f}%")
print("="*90)

# Save for later cells
globals()['class_metrics'] = class_metrics
globals()['class_correct'] = class_correct
globals()['class_total'] = class_total
globals()['class_predictions'] = class_predictions
globals()['class_confidences'] = class_confidences

print("\n✅ Per-class analysis complete!")

DETAILED PER-CLASS ANALYSIS

📊 Analyzing 7 classes in detail...




PER-CLASS PERFORMANCE REPORT

Category      Test Imgs    Top-1    Top-5   Avg Conf Status
------------------------------------------------------------------------------------------
cardboard           236    98.3%   100.0%      98.6%  🌟 Excellent
glass               354    97.7%   100.0%      97.4%  🌟 Excellent
metal               354    93.2%   100.0%      96.4%  ✅ Great
organic             354    99.2%   100.0%      98.9%  🌟 Excellent
paper               472    92.2%   100.0%      94.8%  ✅ Great
plastic             944    93.1%   100.0%      96.3%  ✅ Great
trash               354    91.8%   100.0%      95.2%  ✅ Great
------------------------------------------------------------------------------------------
OVERALL           3,068    94.5%   100.0%

✅ Per-class analysis complete!


In [ ]:

print("CONFUSION ANALYSIS")


print(f"\n📊 Most Common Misclassifications:\n")

confusion_pairs = []

for true_cat in sorted(class_names):
    if true_cat not in class_predictions or class_total[true_cat] == 0:
        continue
    
    predictions = class_predictions[true_cat].copy()
    predictions.pop(true_cat, None)  # Remove correct predictions
    
    if predictions:
        # Find most common mistake
        most_confused = max(predictions.items(), key=lambda x: x[1])
        confusion_rate = (most_confused[1] / class_total[true_cat]) * 100
        
        if confusion_rate > 1:  # Show if >1% confusion
            confusion_pairs.append((true_cat, most_confused[0], most_confused[1], confusion_rate))

# Sort by confusion rate
confusion_pairs.sort(key=lambda x: x[3], reverse=True)

if confusion_pairs:
    print(f"{'True Class':<12} {'→'} {'Predicted As':<12} {'Count':>8} {'Rate':>8}")
    print("-"*70)
    
    for true_cat, pred_cat, count, rate in confusion_pairs:
        print(f"{true_cat:<12} → {pred_cat:<12} {count:>8}x {rate:>7.1f}%")
    
    print("\n💡 These are the class pairs the model confuses most")
else:
    print("✅ No significant confusion between classes!")

# Display confusion matrix
from IPython.display import Image, display

# Find confusion matrix from latest validation
val_dirs = sorted(Path('model_work/runs/classify').glob('val*'), reverse=True)

confusion_img = None
for val_dir in val_dirs:
    candidate = val_dir / 'confusion_matrix_normalized.png'
    if candidate.exists():
        confusion_img = candidate
        break

if confusion_img:
    print("\n" + "="*70)
    print("CONFUSION MATRIX (NORMALIZED)")
    
    print(f"\nLocation: {confusion_img}\n")
    display(Image(filename=str(confusion_img)))
else:
    print("\n⚠️ Confusion matrix image not found")

# Save confusion pairs
globals()['confusion_pairs'] = confusion_pairs

CONFUSION ANALYSIS

📊 Most Common Misclassifications:

True Class   → Predicted As    Count     Rate
----------------------------------------------------------------------
trash        → paper              15x     4.2%
paper        → plastic            15x     3.2%
plastic      → paper              22x     2.3%
metal        → paper               8x     2.3%
glass        → plastic             6x     1.7%
cardboard    → paper               3x     1.3%

💡 These are the class pairs the model confuses most

⚠️ Confusion matrix image not found


In [ ]:
import cv2
import matplotlib.pyplot as plt
import random


print("SAMPLE PREDICTIONS FROM EACH CLASS")


# Show 3 samples per class
fig, axes = plt.subplots(3, 7, figsize=(21, 9))

for col, cat in enumerate(sorted(class_names)):
    cat_path = test_path / cat
    
    if not cat_path.exists():
        continue
    
    images = list(cat_path.glob('*.jpg')) + list(cat_path.glob('*.png'))
    
    if len(images) == 0:
        continue
    
    # Take 3 random samples
    samples = random.sample(images, min(3, len(images)))
    
    for row, img_path in enumerate(samples):
        img = cv2.imread(str(img_path))
        
        if img is not None:
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            # Predict
            pred = model(img_path, verbose=False)
            pred_class = pred[0].names[pred[0].probs.top1]
            confidence = pred[0].probs.top1conf.item()
            
            # Display
            axes[row, col].imshow(img_rgb)
            axes[row, col].axis('off')
            
            # Color based on correctness
            is_correct = (pred_class == cat)
            color = 'green' if is_correct else 'red'
            symbol = '✓' if is_correct else '✗'
            
            title = f"{symbol} {cat}\n→ {pred_class}\n{confidence:.0%}"
            axes[row, col].set_title(title, fontsize=8, color=color, fontweight='bold')

plt.suptitle('Sample Predictions Per Class', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n✅ Sample predictions shown")
print("   🟢 Green = Correct")
print("   🔴 Red = Incorrect")

SAMPLE PREDICTIONS FROM EACH CLASS


<Figure size 2100x900 with 21 Axes>


✅ Sample predictions shown
   🟢 Green = Correct
   🔴 Red = Incorrect


In [ ]:
import json
from datetime import datetime

print("\n" + "="*70)
print("🎉 FINAL EVALUATION REPORT")


print(f"\n📊 OVERALL PERFORMANCE:")
print(f"   Test Images:       {total_images:,}")
print(f"   Top-1 Accuracy:    {overall_top1:.2f}%")
print(f"   Top-5 Accuracy:    {overall_top5:.2f}%")
print(f"   Total Correct:     {total_correct:,}")
print(f"   Total Errors:      {total_images - total_correct:,} ({(total_images - total_correct)/total_images*100:.1f}%)")

print(f"\n🏆 BEST PERFORMING CLASSES:")
best_classes = sorted(class_metrics.items(), key=lambda x: x[1]['top1_accuracy'], reverse=True)[:3]
for i, (cat, metrics) in enumerate(best_classes, 1):
    print(f"   {i}. {cat:12s}: {metrics['top1_accuracy']:.2f}% (avg conf: {metrics['avg_confidence']:.1f}%)")

print(f"\n📉 CLASSES NEEDING ATTENTION:")
worst_classes = sorted(class_metrics.items(), key=lambda x: x[1]['top1_accuracy'])[:3]
for i, (cat, metrics) in enumerate(worst_classes, 1):
    print(f"   {i}. {cat:12s}: {metrics['top1_accuracy']:.2f}% (avg conf: {metrics['avg_confidence']:.1f}%)")

print(f"\n💪 MODEL STRENGTHS:")
excellent = [cat for cat, m in class_metrics.items() if m['top1_accuracy'] >= 95]
great = [cat for cat, m in class_metrics.items() if 90 <= m['top1_accuracy'] < 95]

if excellent:
    print(f"   • Excellent (≥95%): {', '.join(excellent)}")
if great:
    print(f"   • Great (90-95%): {', '.join(great)}")
print(f"   • Top-5 accuracy: {overall_top5:.1f}% (perfect!)")
print(f"   • Overall accuracy exceeds 90% threshold")

print(f"\n🔍 CONFUSION PATTERNS:")
if confusion_pairs:
    print(f"   Top 3 confusion pairs:")
    for i, (true_cat, pred_cat, count, rate) in enumerate(confusion_pairs[:3], 1):
        print(f"   {i}. {true_cat} → {pred_cat}: {count}x ({rate:.1f}%)")
else:
    print(f"   • No significant confusion!")

print(f"\n✅ DEPLOYMENT READINESS:")
min_accuracy = min(m['top1_accuracy'] for m in class_metrics.values())

if overall_top1 >= 90 and min_accuracy >= 85:
    print(f"   🟢 PRODUCTION READY ✓")
    print(f"   • All classes ≥85% accuracy")
    print(f"   • Overall accuracy: {overall_top1:.1f}%")
    print(f"   • Lowest class: {min_accuracy:.1f}%")
    deployment_status = "PRODUCTION_READY"
elif overall_top1 >= 85:
    print(f"   🟡 READY WITH MONITORING")
    print(f"   • Monitor performance on weak classes")
    deployment_status = "READY_WITH_MONITORING"
else:
    print(f"   🔴 NEEDS IMPROVEMENT")
    deployment_status = "NEEDS_IMPROVEMENT"

# Create comprehensive report
report = {
    'metadata': {
        'model_path': str(best_model_path),
        'model_type': 'YOLOv8s-cls',
        'evaluation_date': datetime.now().isoformat(),
        'dataset': str(dataset_path),
        'deployment_status': deployment_status
    },
    'overall_metrics': {
        'total_test_images': int(total_images),
        'top1_accuracy': float(overall_top1 / 100),
        'top5_accuracy': float(overall_top5 / 100),
        'total_correct': int(total_correct),
        'total_errors': int(total_images - total_correct),
        'error_rate': float((total_images - total_correct) / total_images)
    },
    'per_class_metrics': {
        cat: {
            'test_images': int(metrics['total']),
            'correct_predictions': int(metrics['correct']),
            'top1_accuracy': float(metrics['top1_accuracy'] / 100),
            'top5_accuracy': float(metrics['top5_accuracy'] / 100),
            'avg_confidence': float(metrics['avg_confidence'] / 100)
        }
        for cat, metrics in class_metrics.items()
    },
    'confusion_analysis': [
        {
            'true_class': true_cat,
            'predicted_as': pred_cat,
            'count': int(count),
            'confusion_rate': float(rate / 100)
        }
        for true_cat, pred_cat, count, rate in confusion_pairs
    ],
    'disposal_mapping': config.get('disposal_info', {})
}

# Save JSON report
report_path = Path('models/evaluation_report.json')
report_path.parent.mkdir(exist_ok=True)

with open(report_path, 'w') as f:
    json.dump(report, f, indent=2)

print(f"\n💾 Comprehensive report saved:")
print(f"   {report_path}")
print(f"   Size: {report_path.stat().st_size / 1024:.1f} KB")

# Also save a summary text file
summary_path = Path('models/evaluation_summary.txt')

with open(summary_path, 'w') as f:
    f.write("="*70 + "\n")
    f.write("WASTE CLASSIFIER - EVALUATION SUMMARY\n")
    f.write("="*70 + "\n\n")
    f.write(f"Model: YOLOv8s-cls\n")
    f.write(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"Overall Accuracy: {overall_top1:.2f}%\n")
    f.write(f"Deployment Status: {deployment_status}\n\n")
    
    f.write("Per-Class Performance:\n")
    f.write("-"*70 + "\n")
    for cat in sorted(class_names):
        if cat in class_metrics:
            m = class_metrics[cat]
            f.write(f"  {cat:12s}: {m['top1_accuracy']:5.1f}% (conf: {m['avg_confidence']:5.1f}%)\n")

print(f"   {summary_path}")

print("\n" + "="*70)
print("🎉 EVALUATION COMPLETE!")


print("\n📋 Summary:")
print(f"   ✅ Model achieves {overall_top1:.1f}% accuracy")
print(f"   ✅ All {len(class_names)} classes evaluated")
print(f"   ✅ Ready for deployment: {deployment_status}")
print(f"   ✅ Reports saved in models/ directory")

print("\n🚀 Next Steps:")
print("   1. Review confusion matrix and sample predictions")
print("   2. Integrate model with Flask backend")
print("   3. Connect to React frontend")
print("   4. Deploy to production!")


🎉 FINAL EVALUATION REPORT

📊 OVERALL PERFORMANCE:
   Test Images:       3,068
   Top-1 Accuracy:    94.46%
   Top-5 Accuracy:    100.00%
   Total Correct:     2,898
   Total Errors:      170 (5.5%)

🏆 BEST PERFORMING CLASSES:
   1. organic     : 99.15% (avg conf: 98.9%)
   2. cardboard   : 98.31% (avg conf: 98.6%)
   3. glass       : 97.74% (avg conf: 97.4%)

📉 CLASSES NEEDING ATTENTION:
   1. trash       : 91.81% (avg conf: 95.2%)
   2. paper       : 92.16% (avg conf: 94.8%)
   3. plastic     : 93.11% (avg conf: 96.3%)

💪 MODEL STRENGTHS:
   • Excellent (≥95%): cardboard, glass, organic
   • Great (90-95%): metal, paper, plastic, trash
   • Top-5 accuracy: 100.0% (perfect!)
   • Overall accuracy exceeds 90% threshold

🔍 CONFUSION PATTERNS:
   Top 3 confusion pairs:
   1. trash → paper: 15x (4.2%)
   2. paper → plastic: 15x (3.2%)
   3. plastic → paper: 22x (2.3%)

✅ DEPLOYMENT READINESS:
   🟢 PRODUCTION READY ✓
   • All classes ≥85% accuracy
   • Overall accuracy: 94.5%
   • Lowest c